In [5]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("data/")

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

#__change
#Here it is required to load our custom images from data/style_transfer/x and data/style_transfer/y

height = 128
width = 128
pixels = 3
resize_frac=0.3

#code to load images from the folder style_transfer
import glob
import cv2
import os


def load_images_from_folder(folder, images, index):

    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.resize(img, (height, width))
            img = cv2.resize(img,(0,0), fx=resize_frac, fy=resize_frac)
            images[index]=img
            index = index + 1
    return images

x_front = [[]]*1
x_fabric = [[]]*57
y_front = [[]]*57
#print(np.shape(x_front))
#print(np.shape(x_fabric))
#print(np.shape(y_front))
x_front = load_images_from_folder ("D:/deeplearning/cloudxlab/style_transfer/ml-master/deep_learning/data/style_transfer/x/base/front", x_front, 0)
x_fabric = load_images_from_folder ("D:/deeplearning/cloudxlab/style_transfer/ml-master/deep_learning/data/style_transfer/x/fabric", x_fabric, 0)
y_front = load_images_from_folder ("D:/deeplearning/cloudxlab/style_transfer/ml-master/deep_learning/data/style_transfer/y/front", y_front, 0)

#for img in glob.glob("data/x/base/front/*.png"):
#    cv_img = cv2.imread(img)
    
#x_front = [cv2.imread(file) for file in glob.glob("data/x/base/front/*.png")]
#x_fabric = [cv2.imread(file) for file in glob.glob("data/x/fabric/*.jpg")]
#y_front = [cv2.imread(file) for file in glob.glob("data/y/front/*.png")]

#myx-front = cv2.imread("data/x/base/front/x_main_front.png")
print(np.shape(x_front))
print(np.shape(x_fabric))
print(np.shape(y_front))

height = np.int32(height*resize_frac)
width = np.int32(width*resize_frac)

print("height:", height)
print("width:", width)

(1, 38, 38, 3)
(57, 38, 38, 3)
(57, 38, 38, 3)
height: 38
width: 38


In [6]:
x_front = np.array(x_front, dtype=np.float32)/255
x_fabric = np.array(x_fabric, dtype=np.float32)/255
y_front = np.array(y_front, dtype=np.float32)/255


def image_conv_1D2D (img, h, w, p):
    image = np.zeros((h, w, p))
    #print(np.shape(image))
    #print(np.shape(img))
    for i in range(0, h):
        for j in range(0, w):
            #print(i,j)
            image[i,j,:] = img[w*i+j, : ];
    return image

def image_conv_2D1D (image, h, w, p):
    img = np.zeros((h*w, p))
    #print(np.shape(image))
    #print(np.shape(img))
    for i in range(0, h):
        for j in range(0, w):
            #print(i,j)
            img[w*i+j, : ] = image[i,j,:]
    
    return img

def input_multiplex(image1, image2):
    #todo: pass lengths of the images so as to make the function generic
    l1= l2 = height*width
    p = pixels
    im = np.zeros((2*l1, p))
    for i in range (0, l1):
        #print(i)
        im[2*i, :] = image1[i,:]
        im[2*i+1, :] = image2[i,:]
    
    return im

def unroll_RGB(image, length, pixels):
    im = np.zeros((length*pixels))
    for i in range(0, length):
        im[3*i+0] = image[i, 0]
        im[3*i+1] = image[i, 1]
        im[3*i+2] = image[i, 2]
    
    return im

    
    
#img = image_conv_2D1D(x_front[0], 128, 128, 3)
#print(np.shape(img))

#image = image_conv_1D2D(img, 128, 128, 3)
#print(np.shape(image))

#print(x_front[0].shape)
x_front_linear = image_conv_2D1D(x_front[0], height, width, pixels)


xt = [[]]*57
yt = [[]]*57
x_total = [[]]*57
y_total = [[]]*57
#print(np.shape(x_front_linear))
for i in range (0, 57):
    x_fabric_linear = image_conv_2D1D(x_fabric[i], height, width, pixels)
    xt[i] = input_multiplex(x_front_linear, x_fabric_linear)
    #print(i)
    x_total[i] = unroll_RGB(xt[i], 2*height*width, pixels)
    yt[i] = image_conv_2D1D(y_front[i], height, width, pixels)
    y_total[i] = unroll_RGB(yt[i], height*width, pixels)


print(np.shape(xt))
print(np.shape(yt))

print(np.shape(x_total))
print(np.shape(y_total))

(57, 2888, 3)
(57, 1444, 3)
(57, 8664)
(57, 4332)


In [7]:
# Divide the training set 57 inputs into train, valid and test

#x_train = x_total[0:34][:]
x_train, x_valid, x_test = x_total[:34][:], x_total[34:45][:], x_total[45:57][:]
y_train, y_valid, y_test = y_total[:34][:], y_total[34:45][:], y_total[45:57][:]

print(np.shape(x_train))
print(np.shape(x_valid))
print(np.shape(x_test))

print(np.shape(y_train))
print(np.shape(y_valid))
print(np.shape(y_test))

(34, 8664)
(11, 8664)
(12, 8664)
(34, 4332)
(11, 4332)
(12, 4332)


In [8]:
#__change
#n_inputs = 28 * 28  # MNIST
#n_hidden1 = 300
#n_hidden2 = 100
#n_outputs = 10

n_inputs = 2*pixels*height * width  # Style_transfer
n_outputs = pixels*height*width     #no. of pixels (3 pixels for RGB) for 128*128 image

n_hidden1 = n_inputs      # keeping first hidden layer as 4000 neurons
n_hidden2 = n_outputs       # keeping second hidden layer as 400 neurons


In [9]:
print("inputs: ", n_inputs)
print("outputs: ", n_outputs)
print("hidden layer1: ", n_hidden1)
print("hiddden layer2: ", n_hidden2)

inputs:  8664
outputs:  4332
hidden layer1:  8664
hiddden layer2:  4332


In [10]:
reset_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.float32, shape=(None), name="y")

In [12]:
#with tf.variable_scope("model", reuse=True):
with tf.variable_scope(tf.get_variable_scope()) as scope:
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    outputs = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [13]:
loss = tf.reduce_mean(tf.square(outputs - y))

In [14]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [15]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [28]:
#x = x_train[1:2]
#print(np.shape(x))
print(np.shape(x_train)[0])
    

34


In [33]:
n_epochs = 10
batch_size = 1
num_examples = np.shape(x_train)[0]
print("num_examples = ", num_examples)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(num_examples // batch_size):
            X_batch = x_train[iteration:iteration+batch_size]
            y_batch = y_train[iteration:iteration+batch_size]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 2 == 0:
            X_valid = x_valid[epoch:epoch+1]
            Y_valid = y_valid[epoch:epoch+1]
            accuracy = loss
            acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_test = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
            print(epoch, "Batch accuracy:", acc_train, "Validation accuracy:", acc_test)

    save_path = saver.save(sess, "model_ckps/my_model_final.ckpt")

num_examples =  34
0 Batch accuracy: 0.33218 Validation accuracy: 0.478375
2 Batch accuracy: 0.140259 Validation accuracy: 0.19602
4 Batch accuracy: 0.0495633 Validation accuracy: 0.0536372
6 Batch accuracy: 0.02715 Validation accuracy: 0.0212123
8 Batch accuracy: 0.0220358 Validation accuracy: 0.0156844
